In [28]:
import bn
import re
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os 
import numpy as np
from PIL import ImageFilter, ImageEnhance
from google import genai
from dotenv import load_dotenv

load_dotenv()

client = genai.Client()


In [29]:

#extract data pdf to text file
custom_config = r'--oem 3'

def enhance_image_for_ocr(image):
    gray = image.convert('L')
    contrast_enhancer = ImageEnhance.Contrast(gray)
    contrast = contrast_enhancer.enhance(2.0)
    sharp = contrast.filter(ImageFilter.SHARPEN)
    return sharp


def pdf_to_images(pdf_path, dpi=300):
    pages = convert_from_path(pdf_path, dpi=dpi)
    enhanced_pages = []
    for i in range(len(pages)):
        if i>=4 and i<=18:
            crop = pages[i].crop((0, 300, pages[i].width, pages[i].height-400))
            enhanced_page = enhance_image_for_ocr(crop)
            enhanced_pages.append(enhanced_page)
    return enhanced_pages

def extract_text_from_images(images):
    full_text = ""
    for i, img in enumerate(images):
        if i == 0:
            pass
        else:
            print(f"Processing page {i+1}...")
            text = pytesseract.image_to_string(img, lang='ben', config=custom_config)  # 'ben' = Bengali
            clean_text = text.replace('\xa0', ' ')  # Remove non-breaking spaces
            clean_text = "\n".join([line.strip() for line in clean_text.splitlines() if line.strip()])
            #clean_text = clean_text.replace('\n', ' ')
            #full_text += f"\n\n--- Page {i+1} ---\n\n{text}"
            full_text += clean_text
    return full_text


In [54]:
pdf_file = "HSC.pdf"  

In [55]:
text_file = "extract_data2.txt"
if os.path.exists(text_file) == False:
    images = pdf_to_images(pdf_file)
    bangla_text = extract_text_from_images(images)
    with open(text_file, "w", encoding="utf-8") as f:
        f.write(bangla_text)

    print("✅ Done! Bangla text saved to 'extract_data2.txt'")
else:
    print("file exist please continue..........")

Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
✅ Done! Bangla text saved to 'extract_data2.txt'


In [32]:
def chunk_and_vectorize_text(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text_content = file.read()

        # --- Chunking ---
        # Tokenize the text into individual sentences
        text_content = text_content.replace("\n"," ")
        sentences = text_content.split("।")
        print(len(sentences))
        sum_sentence = []
        for i in range(0,len(sentences),8):
            chunk = sentences[i:i+8]
            sum_sentence.append("".join(chunk))
        return sum_sentence
    except:
        print("error")

def search_chunks_multiple_keywords(chunks, keywords):
    # Normalize keywords to lowercase
    keywords = [k.lower() for k in keywords]
    
    # Search for any of the keywords in each chunk
    result = []
    for chunk in chunks:
        chunk_lower = chunk.lower()
        if any(k in chunk_lower for k in keywords):
            result.append(chunk)
    return result


In [33]:
file_path = "extract_data2.txt"
chunks = chunk_and_vectorize_text(file_path)

371


In [34]:
print(len(chunks))

47


In [35]:
#chack data store in database
# api_usage_examples.py
import requests
import json
import uuid
# Base URL of your FastAPI application
BASE_URL = "http://localhost:8000"

In [36]:
def create_sample_chunk(id,chunk,word, token_count):
    """Create a sample data chunk"""
    sample_data = {
        "chunk_id": id,
        "source_file": "sample_document.pdf",
        "chunk_text": chunk,
        "token_count": token_count,
        "start_unit": 0,
        "end_unit": 46,
        "embedding_model": "text-embedding-ada-002",
        "embedding": f'{word}'
    }
    try:
        response = requests.post(f"{BASE_URL}/chunks/", json=sample_data)
        if response.status_code == 200:
            print("✅ Chunk created successfully!")
            chunk_data = response.json()
            print(f"Created chunk with ID: {chunk_data['id']}")
            return chunk_data['id']
        else:
            print(f"❌ Error creating chunk: {response.text}")
            return None
    except:
        print("existing data.....")



for i, ch in enumerate(chunks):
    st=bn.remove_stopwords(ch)
    words_ch = bn.tokenizer(st) # or bn.tokenizer(text, 'word')
    token_count = len(words_ch)
    chunk_id = create_sample_chunk(i,ch,words_ch,token_count)

existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....
existing data.....


In [44]:
def output_get(content,question):
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=f'{content} read this content and give {question} this question answer. only give answer',
    )
    return response.text

In [47]:
question = "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"

In [48]:
st=bn.remove_stopwords(question)
words = bn.tokenizer(st) # or bn.tokenizer(text, 'word')
found_chunks = search_chunks_multiple_keywords(chunks, words)
result_al = []
for i, chunk in enumerate(found_chunks):
    result_al.append(chunk)
result = output_get(result_al,question)
print(result)

১৫ বছর


In [50]:
#data get from dataset 
import requests

# Get all chunk texts
response = requests.get("http://localhost:8001/chunks/texts")
chunk_texts = response.json()
list_chunk= []
for chunk in chunk_texts:
    list_chunk.append(chunk['chunk_text'])
#print(list_chunk)

In [51]:
#get all data
def get_all_chunks():
    """Get all data chunks"""
    response = requests.get(f"{BASE_URL}/chunks/")
    if response.status_code == 200:
        chunks = response.json()
        print(chunks)
        print(f"✅ Retrieved {len(chunks)} chunks")
        for chunk in chunks:
            print(f"  - ID: {chunk['id']}, Source: {chunk['source_file']}")
        return chunks
    else:
        print(f"❌ Error retrieving chunks: {response.text}")
        return []

In [53]:
def generate_bot_response(list_chunk,user_message: str) -> str:
    user_message = user_message.lower().strip()
    print(user_message)
    st=bn.remove_stopwords(user_message)
    words = bn.tokenizer(st) # or bn.tokenizer(text, 'word')
    found_chunks = search_chunks_multiple_keywords(list_chunk, words)
    result_al = []
    for i, chunk in enumerate(found_chunks):
        result_al.append(chunk)
    result = output_get(result_al,user_message)
    print(result)
    return result
question_ai = "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
result = generate_bot_response(list_chunk,question_ai)

বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
বিয়ের প্রস্তাব চলাকালীন কল্যাণীর বয়স ছিল পনেরো।
